In [1]:
#Code to download all needed libraries - needs to be run every time

from __future__ import division #used in Python2.x so the division btw integers result in float, default in python3
from pyomo.environ import * #library for modeling and optimisation
import argparse
from pyomo.opt import SolverStatus, TerminationCondition #provide status of solvers
import pandas as pd #data manipulation library
import numpy as np 
import pickle
import random
import sqlite3
import copy
import random
import csv

import warnings
warnings.filterwarnings('ignore')

INTEREST = 0.03 #interest rate

scen = 500 #number of scenarios for stochastic optimization


path = "C:/VoI/"  #specify the path to the working folder

In [52]:
#This code adds solutions (mngt schedules) to the reference data 

#Adjust the paths to output files from optimization

# Load the reference output file
df1 = pd.read_csv(path+"Outputs/Reference.csv")

# Load the outputs from other approaches
df2 = pd.read_csv(path+"Outputs/direct.csv")

df3 = pd.read_csv(path+"Outputs/directDAP.csv")

df4 = pd.read_csv(path+"Outputs/Height_dif_DAP.csv")

df5 = pd.read_csv(path+"Outputs/Height_dif.csv")

# Extract the 'value' column that stores the decision and save to the reference file

df1['value_direct'] = df2['value']
df1['value_direct_DAP'] = df3['value']
df1['value_height_dif_DAP'] = df4['value']
df1['value_height_dif'] = df5['value']

# Save the modified dataframe to a new csv file
df1.to_csv((path+"Outputs/REF_merged_target1.csv"), index=False) #adjust the path

In [3]:
data = pd.read_csv(path+"Outputs/REF_merged_target1.csv")

In [68]:
#re-computing NPV, CHSI and EndVolume using reference data and decision from another model

data.set_index(['id', 'branch', 'year', 'SCENARIO'], inplace=True)

#replace "value" in the calculation with value from a needed approach:
#value_direct
#value_direct_DAP
#value_height_dif_DAP
#value_height_dif

# NPV
data['NPV_ref'] = data[data.index.get_level_values('year') == 10]['NPV'] * data['value'] * data["area"]
NPV_ref_value = data['NPV_ref'].sum() / scen
print("NPV")
print(NPV_ref_value)

#End Volume
data['EndV_ref'] = data[data2.index.get_level_values('year') == 10]['V_end'] * data['value'] * data["area"]
EndV_ref_value = data['EndV_ref'].sum() / scen
print("End volume")
print(EndV_ref_value)

#HSI
data["CHSI_ref"] = 1 - ((1-data["LSWP"])*(1-data["TTWP"])*(1-data["LTT"])*(1-data["CAP"])*(1-data["HAZ"]))
data["CHSI_ref"] = data["CHSI_ref"]* data['value'] * data["area"]
HSI_ref_value = data["CHSI_ref"].sum() / scen
print("HSI")
print(HSI_ref_value)

NPV
3425628.07664735
End volume
18441.73190557803
HSI
146.90115638373834


In [71]:
#code for CVaR down computation

#value_direct
#value_direct_DAP
#value_height_dif_DAP
#value_height_dif



data['INC_1'] = data['INC'] * data['value_height_dif_DAP'] * data["area"]

total_result = 0
tot_deviation = 0

for i in range(1, 11):
    sum_by_year_ref = data[data.index.get_level_values('year') == i].groupby('SCENARIO')['INC_1'].sum().sort_values(ascending=False)
    #print(sum_by_year_ref)
    tot_deviation = 0
    for k in range (400,500): #range has to be adjusted depending on number of scenarios and confidence interval
        deviation = 600000 - sum_by_year_ref.iloc[k]
        
        if deviation > 0:
            tot_deviation += deviation 
        

    result = tot_deviation * 0.01 #the constrant has to be adjusted depending on on number of scenarios
    #print(result)
    
    if result > 0:
        total_result += result

CVaR_down_ref_value = total_result

print("CVaR_down_ref")
print(CVaR_down_ref_value)

CVaR_down_ref
668822.4626395619


In [72]:
#code for CVaR up computation

#value_direct
#value_direct_DAP
#value_height_dif_DAP
#value_height_dif



data['INC_1'] = data['INC'] * data['value_height_dif_DAP'] * data["area"]

total_result = 0 
tot_deviation = 0

for i in range(1, 11):
    sum_by_year_ref = data[data.index.get_level_values('year') == i].groupby('SCENARIO')['INC_1'].sum().sort_values(ascending=True)
    tot_deviation = 0
    for k in range (400,500): #range has to be adjusted depending on number of scenarios and confidence interval
        deviation = 800000 - sum_by_year_ref.iloc[k]
        
        if deviation < 0:
            tot_deviation += deviation 
        

    result = tot_deviation * 0.01 #the constrant has to be adjusted depending on on number of scenarios
    #print(result)
    
    if result < 0:
        total_result += result

CVaR_up_ref_value = - total_result

print("CVaR_up_ref")
print(CVaR_up_ref_value)

CVaR_up_ref
0


In [10]:
#Input the min and max values from the reference dataset

NPV_min = 2575781.703
NPV_max = 4463200.741

HSI_min = 72.31455707
HSI_max = 205.0860097

CVAR_down_min = 0.00
CVAR_down_max = 5974176.84

CVAR_up_min = 0.00
CVAR_up_max = 3712628.792

EndVol_min = 6341.459545
EndVol_max = 33979.00729

In [64]:
#Input decision-makers targets

NPV_target = 3000000
CVAR_down_target = 0
CVAR_up_target = 0
HSI_target = 200
EndVol_target = 23000

In [73]:
#Code for distances from the target computation for applied output

print((NPV_ref_value - NPV_target)/(NPV_max-NPV_min)) 
print((CVAR_down_target - CVaR_down_ref_value)/(CVAR_down_max - CVAR_down_min))
print((CVAR_up_target -CVaR_up_ref_value)/(CVAR_up_max - CVAR_up_min)) 
print((HSI_ref_value - HSI_target)/(HSI_max - HSI_min)) 
print((EndV_ref_value - EndVol_target)/(EndVol_max - EndVol_min))  

0.13616086495806068
-0.11195223719550322
0.0
-0.1195803728957148
-0.10833287185121253
